In [1]:
import keras.backend as K
from perso import *
import sys
import pickle

Using TensorFlow backend.
/home/jean/anaconda3/envs/kerasenv/lib/python3.6/site-packages/matplotlib/font_manager.py:229: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [2]:
#Pose imports
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, Dense, Activation, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
import scipy
import math
from PIL import Image
from scipy.ndimage.filters import gaussian_filter
%matplotlib inline
import cv2
import matplotlib
import pylab as plt
import numpy as np

sys.path.append('../PoseNet/keras_Realtime_Multi-Person_Pose_Estimation-master/')
import util
from config_reader import config_reader

def relu(x): 
    return Activation('relu')(x)

def conv(x, nf, ks, name):
    x1 = Conv2D(nf, (ks, ks), padding='same', name=name)(x)
    return x1

def pooling(x, ks, st, name):
    x = MaxPooling2D((ks, ks), strides=(st, st), name=name)(x)
    return x

def vgg_block(x):
     
    # Block 1
    x = conv(x, 64, 3, "conv1_1")
    x = relu(x)
    x = conv(x, 64, 3, "conv1_2")
    x = relu(x)
    x = pooling(x, 2, 2, "pool1_1")

    # Block 2
    x = conv(x, 128, 3, "conv2_1")
    x = relu(x)
    x = conv(x, 128, 3, "conv2_2")
    x = relu(x)
    x = pooling(x, 2, 2, "pool2_1")
    
    # Block 3
    x = conv(x, 256, 3, "conv3_1")
    x = relu(x)    
    x = conv(x, 256, 3, "conv3_2")
    x = relu(x)    
    x = conv(x, 256, 3, "conv3_3")
    x = relu(x)    
    x = conv(x, 256, 3, "conv3_4")
    x = relu(x)    
    x = pooling(x, 2, 2, "pool3_1")
    
    # Block 4
    x = conv(x, 512, 3, "conv4_1")
    x = relu(x)    
    x = conv(x, 512, 3, "conv4_2")
    x = relu(x)    
    
    # Additional non vgg layers
    x = conv(x, 256, 3, "conv4_3_CPM")
    x = relu(x)
    x = conv(x, 128, 3, "conv4_4_CPM")
    x = relu(x)
    
    return x

def stage1_block(x, num_p, branch):
    
    # Block 1        
    x = conv(x, 128, 3, "conv5_1_CPM_L%d" % branch)
    x = relu(x)
    x = conv(x, 128, 3, "conv5_2_CPM_L%d" % branch)
    x = relu(x)
    x = conv(x, 128, 3, "conv5_3_CPM_L%d" % branch)
    x = relu(x)
    x = conv(x, 512, 1, "conv5_4_CPM_L%d" % branch)
    x = relu(x)
    x = conv(x, num_p, 1, "conv5_5_CPM_L%d" % branch)
    
    return x

def stageT_block(x, num_p, stage, branch):
        
    # Block 1        
    x = conv(x, 128, 7, "Mconv1_stage%d_L%d" % (stage, branch))
    x = relu(x)
    x = conv(x, 128, 7, "Mconv2_stage%d_L%d" % (stage, branch))
    x = relu(x)
    x = conv(x, 128, 7, "Mconv3_stage%d_L%d" % (stage, branch))
    x = relu(x)
    x = conv(x, 128, 7, "Mconv4_stage%d_L%d" % (stage, branch))
    x = relu(x)
    x = conv(x, 128, 7, "Mconv5_stage%d_L%d" % (stage, branch))
    x = relu(x)
    x = conv(x, 128, 1, "Mconv6_stage%d_L%d" % (stage, branch))
    x = relu(x)
    x = conv(x, num_p, 1, "Mconv7_stage%d_L%d" % (stage, branch))
    
    return x

#MTCNN imports
sys.path.append('keras-mtcnn-master/')
import tools_matrix as tools
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
from MTCNN import create_Kao_Onet, create_Kao_Rnet, create_Kao_Pnet

#Mask R-CNN imports
import os
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import numpy.ma as ma
import face_recognition

# Root directory of the project
ROOT_DIR = os.path.abspath("../Mask_RCNN/Mask_RCNN-master/")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco
sys.path.insert(0, '../Mask_RCNN/Mask_RCNN-master/samples')
import demo

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

In [10]:
def get_pose(model, img_path):
    test_image = img_path
    oriImg = cv2.imread(test_image) # B,G,R order
    heatmap_avg = np.zeros((oriImg.shape[0], oriImg.shape[1], 19))
    paf_avg = np.zeros((oriImg.shape[0], oriImg.shape[1], 38))
    
    param, model_params = config_reader()

    multiplier = [x * model_params['boxsize'] / oriImg.shape[0] for x in param['scale_search']]

    for m in range(len(multiplier)):
        scale = multiplier[m]
        imageToTest = cv2.resize(oriImg, (0,0), fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
        imageToTest_padded, pad = util.padRightDownCorner(imageToTest, model_params['stride'], model_params['padValue'])        

        input_img = np.transpose(np.float32(imageToTest_padded[:,:,:,np.newaxis]), (3,0,1,2)) # required shape (1, width, height, channels) 

        output_blobs = model.predict(input_img)

        # extract outputs, resize, and remove padding
        heatmap = np.squeeze(output_blobs[1]) # output 1 is heatmaps
        heatmap = cv2.resize(heatmap, (0,0), fx=model_params['stride'], fy=model_params['stride'], interpolation=cv2.INTER_CUBIC)
        heatmap = heatmap[:imageToTest_padded.shape[0]-pad[2], :imageToTest_padded.shape[1]-pad[3], :]
        heatmap = cv2.resize(heatmap, (oriImg.shape[1], oriImg.shape[0]), interpolation=cv2.INTER_CUBIC)

        paf = np.squeeze(output_blobs[0]) # output 0 is PAFs
        paf = cv2.resize(paf, (0,0), fx=model_params['stride'], fy=model_params['stride'], interpolation=cv2.INTER_CUBIC)
        paf = paf[:imageToTest_padded.shape[0]-pad[2], :imageToTest_padded.shape[1]-pad[3], :]
        paf = cv2.resize(paf, (oriImg.shape[1], oriImg.shape[0]), interpolation=cv2.INTER_CUBIC)
        
        heatmap_avg = heatmap_avg + heatmap / len(multiplier)
        paf_avg = paf_avg + paf / len(multiplier)

    all_peaks = []
    peak_counter = 0

    for part in range(19-1):
        map_ori = heatmap_avg[:,:,part]
        map = gaussian_filter(map_ori, sigma=3)

        map_left = np.zeros(map.shape)
        map_left[1:,:] = map[:-1,:]
        map_right = np.zeros(map.shape)
        map_right[:-1,:] = map[1:,:]
        map_up = np.zeros(map.shape)
        map_up[:,1:] = map[:,:-1]
        map_down = np.zeros(map.shape)
        map_down[:,:-1] = map[:,1:]

        peaks_binary = np.logical_and.reduce((map>=map_left, map>=map_right, map>=map_up, map>=map_down, map > param['thre1']))
        peaks = list(zip(np.nonzero(peaks_binary)[1], np.nonzero(peaks_binary)[0])) # note reverse
        peaks_with_score = [x + (map_ori[x[1],x[0]],) for x in peaks]
        id = range(peak_counter, peak_counter + len(peaks))
        peaks_with_score_and_id = [peaks_with_score[i] + (id[i],) for i in range(len(id))]

        all_peaks.append(peaks_with_score_and_id)
        peak_counter += len(peaks)
    # find connection in the specified sequence, center 29 is in the position 15
    limbSeq = [[2,3], [2,6], [3,4], [4,5], [6,7], [7,8], [2,9], [9,10], \
               [10,11], [2,12], [12,13], [13,14], [2,1], [1,15], [15,17], \
               [1,16], [16,18], [3,17], [6,18]]
    # the middle joints heatmap correpondence
    mapIdx = [[31,32], [39,40], [33,34], [35,36], [41,42], [43,44], [19,20], [21,22], \
              [23,24], [25,26], [27,28], [29,30], [47,48], [49,50], [53,54], [51,52], \
              [55,56], [37,38], [45,46]]
    connection_all = []
    special_k = []
    mid_num = 10

    for k in range(len(mapIdx)):
        score_mid = paf_avg[:,:,[x-19 for x in mapIdx[k]]]
        candA = all_peaks[limbSeq[k][0]-1]
        candB = all_peaks[limbSeq[k][1]-1]
        nA = len(candA)
        nB = len(candB)
        indexA, indexB = limbSeq[k]
        if(nA != 0 and nB != 0):
            connection_candidate = []
            for i in range(nA):
                for j in range(nB):
                    vec = np.subtract(candB[j][:2], candA[i][:2])
                    norm = math.sqrt(vec[0]*vec[0] + vec[1]*vec[1])
                    # failure case when 2 body parts overlaps
                    if norm == 0:
                        continue
                    vec = np.divide(vec, norm)

                    startend = list(zip(np.linspace(candA[i][0], candB[j][0], num=mid_num), \
                                   np.linspace(candA[i][1], candB[j][1], num=mid_num)))

                    vec_x = np.array([score_mid[int(round(startend[I][1])), int(round(startend[I][0])), 0] \
                                      for I in range(len(startend))])
                    vec_y = np.array([score_mid[int(round(startend[I][1])), int(round(startend[I][0])), 1] \
                                      for I in range(len(startend))])

                    score_midpts = np.multiply(vec_x, vec[0]) + np.multiply(vec_y, vec[1])
                    score_with_dist_prior = sum(score_midpts)/len(score_midpts) + min(0.5*oriImg.shape[0]/norm-1, 0)
                    criterion1 = len(np.nonzero(score_midpts > param['thre2'])[0]) > 0.8 * len(score_midpts)
                    criterion2 = score_with_dist_prior > 0
                    if criterion1 and criterion2:
                        connection_candidate.append([i, j, score_with_dist_prior, score_with_dist_prior+candA[i][2]+candB[j][2]])

            connection_candidate = sorted(connection_candidate, key=lambda x: x[2], reverse=True)
            connection = np.zeros((0,5))
            for c in range(len(connection_candidate)):
                i,j,s = connection_candidate[c][0:3]
                if(i not in connection[:,3] and j not in connection[:,4]):
                    connection = np.vstack([connection, [candA[i][3], candB[j][3], s, i, j]])
                    if(len(connection) >= min(nA, nB)):
                        break

            connection_all.append(connection)
        else:
            special_k.append(k)
            connection_all.append([])
    subset = -1 * np.ones((0, 20))
    candidate = np.array([item for sublist in all_peaks for item in sublist])
    for k in range(len(mapIdx)):
        if k not in special_k:
            partAs = connection_all[k][:,0]
            partBs = connection_all[k][:,1]
            indexA, indexB = np.array(limbSeq[k]) - 1

            for i in range(len(connection_all[k])): #= 1:size(temp,1)
                found = 0
                subset_idx = [-1, -1]
                for j in range(len(subset)): #1:size(subset,1):
                    if subset[j][indexA] == partAs[i] or subset[j][indexB] == partBs[i]:
                        subset_idx[found] = j
                        found += 1

                if found == 1:
                    j = subset_idx[0]
                    if(subset[j][indexB] != partBs[i]):
                        subset[j][indexB] = partBs[i]
                        subset[j][-1] += 1
                        subset[j][-2] += candidate[partBs[i].astype(int), 2] + connection_all[k][i][2]
                elif found == 2: # if found 2 and disjoint, merge them
                    j1, j2 = subset_idx
                    print ("found = 2")
                    membership = ((subset[j1]>=0).astype(int) + (subset[j2]>=0).astype(int))[:-2]
                    if len(np.nonzero(membership == 2)[0]) == 0: #merge
                        subset[j1][:-2] += (subset[j2][:-2] + 1)
                        subset[j1][-2:] += subset[j2][-2:]
                        subset[j1][-2] += connection_all[k][i][2]
                        subset = np.delete(subset, j2, 0)
                    else: # as like found == 1
                        subset[j1][indexB] = partBs[i]
                        subset[j1][-1] += 1
                        subset[j1][-2] += candidate[partBs[i].astype(int), 2] + connection_all[k][i][2]

                # if find no partA in the subset, create a new subset
                elif not found and k < 17:
                    row = -1 * np.ones(20)
                    row[indexA] = partAs[i]
                    row[indexB] = partBs[i]
                    row[-1] = 2
                    row[-2] = sum(candidate[connection_all[k][i,:2].astype(int), 2]) + connection_all[k][i][2]
                    subset = np.vstack([subset, row])
    # delete some rows of subset which has few parts occur
    deleteIdx = [];
    for i in range(len(subset)):
        if subset[i][-1] < 4 or subset[i][-2]/subset[i][-1] < 0.4:
            deleteIdx.append(i)
    subset = np.delete(subset, deleteIdx, axis=0)
    colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0], [0, 255, 0], \
          [0, 255, 85], [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], [0, 0, 255], [85, 0, 255], \
          [170, 0, 255], [255, 0, 255], [255, 0, 170], [255, 0, 85]]
    cmap = matplotlib.cm.get_cmap('hsv')

    canvas = cv2.imread(test_image) # B,G,R order

    for i in range(18):
        rgba = np.array(cmap(1 - i/18. - 1./36))
        rgba[0:3] *= 255
        for j in range(len(all_peaks[i])):
            cv2.circle(canvas, all_peaks[i][j][0:2], 4, colors[i], thickness=-1)
    stickwidth = 4
    cur_canvas = np.zeros(canvas.shape, dtype='uint8')
    cur_canvas.fill(255)
    for i in range(17):
        for n in range(len(subset)):
            index = subset[n][np.array(limbSeq[i])-1]
            if -1 in index:
                continue
            Y = candidate[index.astype(int), 0]
            X = candidate[index.astype(int), 1]
            mX = np.mean(X)
            mY = np.mean(Y)
            length = ((X[0] - X[1]) ** 2 + (Y[0] - Y[1]) ** 2) ** 0.5
            angle = math.degrees(math.atan2(X[0] - X[1], Y[0] - Y[1]))
            polygon = cv2.ellipse2Poly((int(mY),int(mX)), (int(length/2), stickwidth), int(angle), 0, 360, 1)
            cv2.fillConvexPoly(cur_canvas, polygon, colors[i])
            canvas = cv2.addWeighted(canvas, 0.4, cur_canvas, 0.6, 0)
        
    return cur_canvas

In [6]:
def detectFace(img, threshold, Pnet, Rnet, Onet):

    caffe_img = (img.copy() - 127.5) / 127.5
    origin_h, origin_w, ch = caffe_img.shape
    scales = tools.calculateScales(img)
    out = []
    t0 = time.time()
    # del scales[:4]

    for scale in scales:
        hs = int(origin_h * scale)
        ws = int(origin_w * scale)
        scale_img = cv2.resize(caffe_img, (ws, hs))
        input = scale_img.reshape(1, *scale_img.shape)
        ouput = Pnet.predict(input)  # .transpose(0,2,1,3) should add, but seems after process is wrong then.
        out.append(ouput)
    image_num = len(scales)
    rectangles = []
    for i in range(image_num):
        cls_prob = out[i][0][0][:, :,
                   1]  # i = #scale, first 0 select cls score, second 0 = batchnum, alway=0. 1 one hot repr
        roi = out[i][1][0]
        out_h, out_w = cls_prob.shape
        out_side = max(out_h, out_w)
        # print('calculating img scale #:', i)
        cls_prob = np.swapaxes(cls_prob, 0, 1)
        roi = np.swapaxes(roi, 0, 2)
        rectangle = tools.detect_face_12net(cls_prob, roi, out_side, 1 / scales[i], origin_w, origin_h, threshold[0])
        rectangles.extend(rectangle)
    rectangles = tools.NMS(rectangles, 0.7, 'iou')

    t1 = time.time()
    print ('time for 12 net is: ', t1-t0, len(rectangles))

    if len(rectangles) == 0:
        return rectangles

    crop_number = 0
    out = []
    predict_24_batch = []
    for rectangle in rectangles:
        crop_img = caffe_img[int(rectangle[1]):int(rectangle[3]), int(rectangle[0]):int(rectangle[2])]
        scale_img = cv2.resize(crop_img, (24, 24))
        predict_24_batch.append(scale_img)
        crop_number += 1

    predict_24_batch = np.array(predict_24_batch)

    out = Rnet.predict(predict_24_batch)

    cls_prob = out[0]  # first 0 is to select cls, second batch number, always =0
    cls_prob = np.array(cls_prob)  # convert to numpy
    roi_prob = out[1]  # first 0 is to select roi, second batch number, always =0
    roi_prob = np.array(roi_prob)
    rectangles = tools.filter_face_24net(cls_prob, roi_prob, rectangles, origin_w, origin_h, threshold[1])
    t2 = time.time()
    print ('time for 24 net is: ', t2-t1, len(rectangles))


    if len(rectangles) == 0:
        return rectangles


    crop_number = 0
    predict_batch = []
    for rectangle in rectangles:
        # print('calculating net 48 crop_number:', crop_number)
        crop_img = caffe_img[int(rectangle[1]):int(rectangle[3]), int(rectangle[0]):int(rectangle[2])]
        scale_img = cv2.resize(crop_img, (48, 48))
        predict_batch.append(scale_img)
        crop_number += 1

    predict_batch = np.array(predict_batch)

    output = Onet.predict(predict_batch)
    cls_prob = output[0]
    roi_prob = output[1]
    pts_prob = output[2]  # index
    # rectangles = tools.filter_face_48net_newdef(cls_prob, roi_prob, pts_prob, rectangles, origin_w, origin_h,
    #                                             threshold[2])
    rectangles = tools.filter_face_48net(cls_prob, roi_prob, pts_prob, rectangles, origin_w, origin_h, threshold[2])
    t3 = time.time()
    print ('time for 48 net is: ', t3-t2, len(rectangles))

    return rectangles

def get_face(img, Pnet, Rnet, Onet):
    threshold = [0.1,0.3,0.3]
    # ret, img = cap.read()
    img = cv2.imread(img)
    rectangles = detectFace(img, threshold, Pnet, Rnet, Onet)
    draw = np.zeros(img.shape)

    for rectangle in rectangles:
        if rectangle is not None:
            print(np.round(np.array(rectangle)))
            W = -int(rectangle[0]) + int(rectangle[2])
            H = -int(rectangle[1]) + int(rectangle[3])
            paddingH = 0.01 * W
            paddingW = 0.02 * H
            for row in range(draw.shape[0]):
                for px in range(draw.shape[1]):
                    if row > int(rectangle[1]+paddingH) and row < int(rectangle[3]-paddingH):
                        if px > int(rectangle[0]-paddingW) and px < int(rectangle[2]+paddingW):
                            draw[row][px] = img[row][px]
    return draw.astype('uint8')[:,:,[2,1,0]]

In [7]:
def crop_imgs(nbr_imgs, images_path, full_path, background_path, persons_path, mask_path, pose_path, keep_all = True, ext='.png'):
    K.clear_session()
    config = InferenceConfig()
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

    # Load weights trained on MS-COCO
    model.load_weights(COCO_MODEL_PATH, by_name=True)
    class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
                   'bus', 'train', 'truck', 'boat', 'traffic light',
                   'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
                   'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
                   'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
                   'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
                   'kite', 'baseball bat', 'baseball glove', 'skateboard',
                   'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
                   'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
                   'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
                   'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
                   'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
                   'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
                   'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
                   'teddy bear', 'hair drier', 'toothbrush']
    f = 0
    for k in range(nbr_imgs):
        img = '{}/{}{}'.format(images_path, k, ext)
        img = skimage.io.imread(img)
        results = model.detect([img])[0]
        masks = results['masks']
        pos = np.where(results['class_ids'] == 1)[0]
        if not keep_all: 
            if len(pos) == 0 : continue
        ttl_mask = np.zeros(img.shape[:-1])
        for p in pos:
            mask = masks[:,:,p].astype('bool')
            ttl_mask = mask + (ttl_mask > 0)
        background = skimage.io.imread(os.path.join(pose_path,str(k)+'.png'))
        person = np.zeros(shape=(img.shape))
        for row in range(background.shape[0]):
            for pixel in range(background.shape[1]):
                if not ttl_mask[row][pixel]:
                    background[row][pixel] = img[row][pixel]
                else: person[row][pixel] = img[row][pixel]
            
        with open(mask_path + '/' + str(f) + '.msk', 'wb') as file:
            pickle.dump(ttl_mask, file)
        img = Image.fromarray(img.astype('uint8'), 'RGB')
        background = Image.fromarray(background.astype('uint8'), 'RGB')
        person = Image.fromarray(person.astype('uint8'), 'RGB')
        img.save('{}/{}.png'.format(full_path, f))
        background.save('{}/{}.png'.format(background_path, f))
        person.save('{}/{}.png'.format(persons_path, f))
        print(k)
        f+=1

In [8]:
def transform_images(images_path, full_path, background_path, persons_path, mask_path, pose_path, keep_all = True, ext='.png',
                    base = 0):
    weights_path = "keras_Realtime_Multi-Person_Pose_Estimation-master/model/keras/model.h5" # orginal weights converted from caffe
    #weights_path = "training/weights.best.h5" # weights tarined from scratch 

    input_shape = (None,None,3)

    img_input = Input(shape=input_shape)

    stages = 6
    np_branch1 = 38
    np_branch2 = 19

    img_normalized = Lambda(lambda x: x / 256 - 0.5)(img_input)  # [-0.5, 0.5]

    # VGG
    stage0_out = vgg_block(img_normalized)

    # stage 1
    stage1_branch1_out = stage1_block(stage0_out, np_branch1, 1)
    stage1_branch2_out = stage1_block(stage0_out, np_branch2, 2)
    x = Concatenate()([stage1_branch1_out, stage1_branch2_out, stage0_out])

    # stage t >= 2
    for sn in range(2, stages + 1):
        stageT_branch1_out = stageT_block(x, np_branch1, sn, 1)
        stageT_branch2_out = stageT_block(x, np_branch2, sn, 2)
        if (sn < stages):
            x = Concatenate()([stageT_branch1_out, stageT_branch2_out, stage0_out])

    model = Model(img_input, [stageT_branch1_out, stageT_branch2_out])
    model.load_weights(weights_path)
    
    nbr_imgs = getLastFromDir(images_path) - base
    for f in range(nbr_imgs):
        img = '{}/{}{}'.format(images_path, f + base,ext)
        print(img)
        canvas = get_pose(model, img)
        print(f, ' ', canvas.shape)
        canvas = Image.fromarray(canvas.astype('uint8'), 'RGB')
        canvas.save('{}/{}.png'.format(pose_path, f))
    
    K.clear_session()
    Pnet = create_Kao_Pnet('keras-mtcnn-master/12net.h5')
    Rnet = create_Kao_Rnet('keras-mtcnn-master/24net.h5')
    Onet = create_Kao_Onet('keras-mtcnn-master/48net.h5')
    
    for f in range(nbr_imgs):
        face = get_face(images_path+'/{}{}'.format(f+base, ext), Pnet, Rnet, Onet)
        pose = skimage.io.imread(pose_path+'/{}.png'.format(f))
        for row in range(face.shape[0]):
            for px in range(face.shape[1]):
                if face[row][px].all() != np.zeros(3).all():
                    pose[row][px] = face[row][px]
        image = Image.fromarray(pose.astype('uint8'), 'RGB')
        image.save(pose_path+'/{}.png'.format(f))
    
    
    K.clear_session()
    config = InferenceConfig()
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

    # Load weights trained on MS-COCO
    model.load_weights(COCO_MODEL_PATH, by_name=True)
    class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
                   'bus', 'train', 'truck', 'boat', 'traffic light',
                   'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
                   'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
                   'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
                   'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
                   'kite', 'baseball bat', 'baseball glove', 'skateboard',
                   'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
                   'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
                   'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
                   'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
                   'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
                   'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
                   'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
                   'teddy bear', 'hair drier', 'toothbrush']
    
    for f in range(nbr_imgs):
        img = '{}/{}{}'.format(images_path, f + base, ext)
        img = skimage.io.imread(img)
        results = model.detect([img])[0]
        masks = results['masks']
        pos = np.where(results['class_ids'] == 1)[0]
        if len(pos) == 0 : continue
        ttl_mask = np.zeros(img.shape[:-1])
        for p in pos:
            mask = masks[:,:,p].astype('bool')
            ttl_mask = mask + (ttl_mask > 0)
        background = skimage.io.imread(os.path.join(pose_path,str(f)+'.png'))
        person = np.zeros(shape=(img.shape))
        for row in range(background.shape[0]):
            for pixel in range(background.shape[1]):
                if not ttl_mask[row][pixel]:
                    background[row][pixel] = np.zeros(3)
                else: person[row][pixel] = img[row][pixel]
            
        with open(mask_path + '/' + str(f) + '.msk', 'wb') as file:
            pickle.dump(ttl_mask, file)
        img = Image.fromarray(img.astype('uint8'), 'RGB')
        background = Image.fromarray(background.astype('uint8'), 'RGB')
        person = Image.fromarray(person.astype('uint8'), 'RGB')
        img.save('{}/{}.png'.format(full_path, f))
        background.save('{}/{}.png'.format(background_path, f))
        person.save('{}/{}.png'.format(persons_path, f))
        print(f)

In [12]:
transform_images('data/128x128/dressed', 'data/128x128/cropped/dressed/full', 'data/128x128/cropped/dressed/backgrounds',
                'data/128x128/cropped/dressed/persons','data/128x128/cropped/dressed/masks','data/128x128/cropped/dressed/poses', ext='.jpg')

data/128x128/dressed/0.jpg
0   (128, 128, 3)
data/128x128/dressed/1.jpg
1   (128, 128, 3)
data/128x128/dressed/2.jpg
found = 2
2   (128, 128, 3)
data/128x128/dressed/3.jpg
found = 2
3   (128, 128, 3)
data/128x128/dressed/4.jpg
4   (128, 128, 3)
data/128x128/dressed/5.jpg
5   (128, 128, 3)
data/128x128/dressed/6.jpg
6   (128, 128, 3)
data/128x128/dressed/7.jpg
7   (128, 128, 3)
data/128x128/dressed/8.jpg
8   (128, 128, 3)
data/128x128/dressed/9.jpg
9   (128, 128, 3)
data/128x128/dressed/10.jpg
10   (128, 128, 3)
data/128x128/dressed/11.jpg
11   (128, 128, 3)
data/128x128/dressed/12.jpg
12   (128, 128, 3)
data/128x128/dressed/13.jpg
13   (128, 128, 3)
data/128x128/dressed/14.jpg
14   (128, 128, 3)
data/128x128/dressed/15.jpg
15   (128, 128, 3)
data/128x128/dressed/16.jpg
16   (128, 128, 3)
data/128x128/dressed/17.jpg
17   (128, 128, 3)
data/128x128/dressed/18.jpg
18   (128, 128, 3)
data/128x128/dressed/19.jpg
19   (128, 128, 3)
data/128x128/dressed/20.jpg
20   (128, 128, 3)
data/128x128/

170   (128, 128, 3)
data/128x128/dressed/171.jpg
171   (128, 128, 3)
data/128x128/dressed/172.jpg
172   (128, 128, 3)
data/128x128/dressed/173.jpg
173   (128, 128, 3)
data/128x128/dressed/174.jpg
174   (128, 128, 3)
data/128x128/dressed/175.jpg
175   (128, 128, 3)
data/128x128/dressed/176.jpg
176   (128, 128, 3)
data/128x128/dressed/177.jpg
177   (128, 128, 3)
data/128x128/dressed/178.jpg
178   (128, 128, 3)
data/128x128/dressed/179.jpg
179   (128, 128, 3)
data/128x128/dressed/180.jpg
180   (128, 128, 3)
data/128x128/dressed/181.jpg
181   (128, 128, 3)
data/128x128/dressed/182.jpg
182   (128, 128, 3)
data/128x128/dressed/183.jpg
183   (128, 128, 3)
data/128x128/dressed/184.jpg
184   (128, 128, 3)
data/128x128/dressed/185.jpg
185   (128, 128, 3)
data/128x128/dressed/186.jpg
186   (128, 128, 3)
data/128x128/dressed/187.jpg
187   (128, 128, 3)
data/128x128/dressed/188.jpg
188   (128, 128, 3)
data/128x128/dressed/189.jpg
189   (128, 128, 3)
data/128x128/dressed/190.jpg
190   (128, 128, 3)


337   (128, 128, 3)
data/128x128/dressed/338.jpg
338   (128, 128, 3)
data/128x128/dressed/339.jpg
339   (128, 128, 3)
data/128x128/dressed/340.jpg
340   (128, 128, 3)
data/128x128/dressed/341.jpg
341   (128, 128, 3)
data/128x128/dressed/342.jpg
342   (128, 128, 3)
data/128x128/dressed/343.jpg
343   (128, 128, 3)
data/128x128/dressed/344.jpg
344   (128, 128, 3)
data/128x128/dressed/345.jpg
345   (128, 128, 3)
data/128x128/dressed/346.jpg
346   (128, 128, 3)
data/128x128/dressed/347.jpg
347   (128, 128, 3)
data/128x128/dressed/348.jpg
found = 2
348   (128, 128, 3)
data/128x128/dressed/349.jpg
349   (128, 128, 3)
data/128x128/dressed/350.jpg
found = 2
350   (128, 128, 3)
data/128x128/dressed/351.jpg
351   (128, 128, 3)
data/128x128/dressed/352.jpg
352   (128, 128, 3)
data/128x128/dressed/353.jpg
353   (128, 128, 3)
data/128x128/dressed/354.jpg
354   (128, 128, 3)
data/128x128/dressed/355.jpg
355   (128, 128, 3)
data/128x128/dressed/356.jpg
found = 2
356   (128, 128, 3)
data/128x128/dresse

504   (128, 128, 3)
data/128x128/dressed/505.jpg
505   (128, 128, 3)
data/128x128/dressed/506.jpg
506   (128, 128, 3)
data/128x128/dressed/507.jpg
507   (128, 128, 3)
data/128x128/dressed/508.jpg
508   (128, 128, 3)
data/128x128/dressed/509.jpg
509   (128, 128, 3)
data/128x128/dressed/510.jpg
510   (128, 128, 3)
data/128x128/dressed/511.jpg
511   (128, 128, 3)
data/128x128/dressed/512.jpg
512   (128, 128, 3)
data/128x128/dressed/513.jpg
513   (128, 128, 3)
data/128x128/dressed/514.jpg
514   (128, 128, 3)
data/128x128/dressed/515.jpg
515   (128, 128, 3)
data/128x128/dressed/516.jpg
516   (128, 128, 3)
data/128x128/dressed/517.jpg
517   (128, 128, 3)
data/128x128/dressed/518.jpg
518   (128, 128, 3)
data/128x128/dressed/519.jpg
519   (128, 128, 3)
data/128x128/dressed/520.jpg
520   (128, 128, 3)
data/128x128/dressed/521.jpg
521   (128, 128, 3)
data/128x128/dressed/522.jpg
522   (128, 128, 3)
data/128x128/dressed/523.jpg
523   (128, 128, 3)
data/128x128/dressed/524.jpg
524   (128, 128, 3)


671   (128, 128, 3)
data/128x128/dressed/672.jpg
672   (128, 128, 3)
data/128x128/dressed/673.jpg
673   (128, 128, 3)
data/128x128/dressed/674.jpg
674   (128, 128, 3)
data/128x128/dressed/675.jpg
675   (128, 128, 3)
data/128x128/dressed/676.jpg
676   (128, 128, 3)
data/128x128/dressed/677.jpg
677   (128, 128, 3)
data/128x128/dressed/678.jpg
678   (128, 128, 3)
data/128x128/dressed/679.jpg
679   (128, 128, 3)
data/128x128/dressed/680.jpg
680   (128, 128, 3)
data/128x128/dressed/681.jpg
681   (128, 128, 3)
data/128x128/dressed/682.jpg
682   (128, 128, 3)
data/128x128/dressed/683.jpg
683   (128, 128, 3)
data/128x128/dressed/684.jpg
684   (128, 128, 3)
data/128x128/dressed/685.jpg
685   (128, 128, 3)
data/128x128/dressed/686.jpg
686   (128, 128, 3)
data/128x128/dressed/687.jpg
687   (128, 128, 3)
data/128x128/dressed/688.jpg
found = 2
688   (128, 128, 3)
data/128x128/dressed/689.jpg
689   (128, 128, 3)
data/128x128/dressed/690.jpg
690   (128, 128, 3)
data/128x128/dressed/691.jpg
691   (128

...

time for 12 net is:  0.22029709815979004 1330
time for 24 net is:  0.1051642894744873 6
time for 48 net is:  0.0038700103759765625 2
[22.  9. 38. 30.  1. 28. 18. 35. 18. 32. 22. 29. 26. 34. 26.]
[ 91.  10. 105.  29.   0.  96.  18. 102.  18.  99.  21.  96.  25. 102.
  25.]
time for 12 net is:  0.5409808158874512 2457
time for 24 net is:  0.1924128532409668 16
time for 48 net is:  0.005875825881958008 3
[118.  83. 126.  94.   0. 121.  89. 126.  88. 123.  91. 122.  93. 125.
  92.]
[101.  80. 110.  91.   0. 105.  85. 109.  85. 107.  88. 105.  90. 109.
  90.]
[ 90.  40. 105.  60.   0.  93.  48.  99.  46.  95.  52.  95.  57. 100.
  55.]
time for 12 net is:  0.044574737548828125 105
time for 24 net is:  0.010229110717773438 0
time for 12 net is:  0.07034635543823242 388
time for 24 net is:  0.03389883041381836 2
time for 48 net is:  0.0033304691314697266 1
[26. 20. 47. 44.  0. 36. 30. 46. 30. 42. 34. 36. 40. 44. 41.]
time for 12 net is:  0.05902743339538574 193
time for 24 net is:  0.01764798

/home/jean/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279

1877
1878
1879
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080


3533
3534
3535
3536
3537
3538
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736


In [ ]:
crop_imgs(4276,'data/128x128/dressed', 'data/128x128/cropped/dressed/full', 'data/128x128/cropped/dressed/backgrounds',
                'data/128x128/cropped/dressed/persons','data/128x128/cropped/dressed/masks','data/128x128/cropped/dressed/poses', 
          keep_all = False, ext='.jpg')

/home/jean/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279

In [33]:
config = ConfigObj('config')

In [34]:
config['param']

{'use_gpu': '1', 'GPUdeviceNumber': '0', 'modelID': '1', 'octave': '3', 'starting_range': '0.8', 'ending_range': '2', 'scale_search': ['0.5', '1', '1.5', '2'], 'thre1': '0.1', 'thre2': '0.05', 'thre3': '0.5', 'min_num': '4', 'mid_num': '10', 'crop_ratio': '2.5', 'bbox_ratio': '0.25'}